Saleh Ahmad
i200605@nu.edu.pk

In [1]:
import numpy as np
from collections import defaultdict
import random

In [2]:
class Bidirectional_Bigram_LM_Roman_Urdu():
    '''
    I have a question, does a bidirectional look for left word when predicting from left and right word when predicting from right separately?
    or does it also look at left of "to be predicted word" and also to the end/right when predicting from left, same for when predicting from right.

    Sipposing this one looks at left only for predicting from left to right and right only when predicting from right to left.
    '''
    def __init__(self):
        self.ngram = 2

    def fit(self,TrainCorpusVocab): #TrainCorpus = all tokens in order of occurence
        Vocabs = defaultdict(int)
        UniqueVocabSize = defaultdict(int)
        VocabSize = defaultdict(int)
        for ngram in range(self.ngram,0,-1):
            IdxToken = 0
            ngrams = defaultdict(int)
            Len_Vocab = len(TrainCorpusVocab)
            while IdxToken < Len_Vocab-ngram+1:
                current_n_gram = TrainCorpusVocab[IdxToken]
                for idx in range(1,ngram):
                    current_n_gram += ' ' + TrainCorpusVocab[IdxToken+idx]
                
                ngrams[current_n_gram] += 1
                IdxToken += 1

            ngrams = {k:v for k,v in ngrams.items() if v > 0}
            Vocabs[ngram] = ngrams
            UniqueVocabSize[ngram] = len(ngrams)
            VocabSize[ngram] = sum(ngrams.values())
        
        self.Vocabs = Vocabs
        self.UniqueVocabSizes = UniqueVocabSize
        self.VocabSizes = VocabSize

    def __Get_Random_Length_Of_Verse(self):
        return 8

    def __Get_Random_Ending_Token(self,e_tokens):
        e_tokens = np.unique(e_tokens)
        return random.choice(e_tokens)
    
    def __Get_Random_Starting_Token(self,s_tokens):
        return random.choice(s_tokens)

    def __Get_Probability_Based_Ending_Tokens(self,s_tokens,Chosen_Starting_Token,e_tokens):
        '''
        Given that a starting token has been chosen randomly, I think that the ending token should be based upon which has the most probability.
        Do I also need a vocab for this? Let's Try.
        '''
        indices = np.array([idx for idx,token in enumerate(s_tokens) if token == Chosen_Starting_Token])
        Ending_Token_Candidates = [e_tokens[idx] for idx in indices]
        unique,count = np.unique(Ending_Token_Candidates,return_counts=True)
        Chosen_Ending_Token = unique[np.argmax(count)]
        return Chosen_Ending_Token
    
    def __Print_Vocab(self):
        print(self.Vocabs)

    def __Get_Next_Word_From_Candidate_Right_To_Left(self,Sequence):
        NGramToUse = self.ngram
        Candidates = [ngram for ngram in self.Vocabs[NGramToUse].keys() if ''.join(ngram.split()[-1]) == Sequence]
                
        if len(Candidates) == 0:
            ToReturn = random.choice(list(self.Vocabs[NGramToUse-1].keys())[:10])
            return ToReturn
        else:
            Probability_Of_Candidates = np.zeros_like(Candidates)
            for idx,candidate in enumerate(Candidates):
                Probability_Of_Candidates[idx] = (self.Vocabs[NGramToUse][candidate] + 1) / (self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram])
            return Candidates[np.argmax(Probability_Of_Candidates)].split()[-2]

    def __Get_Next_Word_From_Candidate_Left_To_Right(self,Sequence):
        ngram_ToUse = self.ngram
        Candidates = [ngram for ngram in self.Vocabs[ngram_ToUse].keys() if ' '.join(ngram.split()[0:-1]) == Sequence]

        if len(Candidates) == 0:
            ToReturn = random.choice(list(self.Vocabs[ngram_ToUse-1].keys())[:10])
            return ToReturn
        else:
            Probs_Of_Candidates = np.zeros_like(Candidates)
            for idxCandidate,candidate in enumerate(Candidates):
                Probs_Of_Candidates[idxCandidate] = (self.Vocabs[ngram_ToUse][candidate] + 1) / (self.VocabSizes[self.ngram] + self.UniqueVocabSizes[self.ngram])
            return Candidates[np.argmax(Probs_Of_Candidates)].split()[-1] #return max prob and the next word

    def predict(self,StartingTokens,EndingTokens): #StartingTokens = list of repetitive tokens, can extract unique and count
        '''
        Prediction format
        1 stanza = 14 verses (1 * 14)
        1 verse = 6 - 8 words
        '''        

        Verse = ''
        for idxVerse in range(0,14):
            StartingTokenOfThisVerse = self.__Get_Random_Starting_Token(StartingTokens)
            EndingTokenOfThisVerse =  self.__Get_Probability_Based_Ending_Tokens(StartingTokens,StartingTokenOfThisVerse,EndingTokens)
            LengthOfVerse = self.__Get_Random_Length_Of_Verse()
            FirstHalf = int(LengthOfVerse/2)-1
            SecondHalf = FirstHalf-1

            VerseLeftToRight = StartingTokenOfThisVerse
            Sequence = StartingTokenOfThisVerse
            #From Left To Right
            for idx in range(0,FirstHalf):
                NextWord = self.__Get_Next_Word_From_Candidate_Left_To_Right(Sequence)
                VerseLeftToRight += ' ' + NextWord
                Sequence = NextWord

            Sequence = EndingTokenOfThisVerse
            VerseRightToLeft = EndingTokenOfThisVerse
            #From Right To Left
            for idx in range(0,SecondHalf):
                PrevWord = self.__Get_Next_Word_From_Candidate_Right_To_Left(Sequence)
                VerseRightToLeft = PrevWord + ' ' + VerseRightToLeft
                Sequence = PrevWord

            Verse += VerseLeftToRight + ' ' + VerseRightToLeft + '\n'
            if idxVerse != 0 and idxVerse % 2 == 1:
                Verse += '\n'
        return Verse

    def Print_Details(self):
        print('All ngrams',self.Vocabs.keys())
        for ngram in self.Vocabs.keys():
            print('ngram:',ngram)
            print('ngrams:',list(self.Vocabs[ngram].keys())[0:10])
            print('priors:',list(self.Priors[ngram].keys())[0:10])


In [3]:
AllTokens = np.load('../Data/All_Tokens_Roman_Urdu.npy')
AllTokens = [token.lower() for token in AllTokens]
EndingTokens = np.load('../Data/Ending_Tokens_Roman_Urdu.npy')
EndingTokens = [token.lower() for token in EndingTokens]
StartingTokens = np.load('../Data/Starting_Tokens_Roman_Urdu.npy')
StartingTokens = [token.lower() for token in StartingTokens]

In [4]:
ObjLM = Bidirectional_Bigram_LM_Roman_Urdu()
ObjLM.fit(AllTokens)
# ObjLM.Print_Details()

In [5]:
Pred = ObjLM.predict(StartingTokens,EndingTokens)
print(Pred)

rehne walo kkhuda ne se aati agay
samnay tairay koochy se aaye nah aaye

deewangi se kuch aisi dil hi nahi
suna hai shab mein kis ne aakhir

taaraaj kawish gham rozgaar hai sun-hwa asad
yeh kis ne ki apna bana hai

ik shore uthaya ghalib se aati agay
qanaat nah kyun kar bana hai par

deewana par yeh kis dil hi nahi
aatish khamosh ki hain dil guzar gaya

yeh kis ne ki apna bana hai
ghalib tujhe hum pay ne ki hain

khumyaza hon is pay nah aaya tha
aah mein le ga hi se nikla


